# Importando Bibliotecas

In [1]:
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
import time
import urllib.request
import pandas as pd

# Seletores

In [2]:
# --- Seletores Globais ---
classe_caixa_aceitar = 'adopt-c-bXGRNs'
id_aceitar = 'adopt-accept-all-button'
CLASSE_PRODUTO_ITEM = 'product-item-info'
CLASSE_PRODUTO_NOME = 'product-item-link'
CLASSE_PRODUTO_PRECO = 'price'
CLASSE_PRODUTO_DESCRICAO = 'product-item-description'

# Código Principal

In [ ]:
class NEO:
    '''
    O método __init__ é um inicializador que não retorna nenhum valor. 
    Seu trabalho é configurar o objeto (self), e não produzir um resultado.
    '''
    def __init__(self) -> None:


        self.base_url = "https://www.neosolar.com.br/loja"
        options = ChromeOptions()
        options.add_argument("--start-maximized")
        
        # Atualização automática do Chromedriver
        try:
            service = Service(ChromeDriverManager().install())
        except ValueError:
            latest_chromedriver_version_url = "https://chromedriver.storage.googleapis.com/LATEST_RELEASE"
            latest_chromedriver_version = urllib.request.urlopen(latest_chromedriver_version_url).read().decode('utf-8')
            service = Service(ChromeDriverManager(version=latest_chromedriver_version).install())

        self.browser = Chrome(service=service, options=options)
        self.wait = WebDriverWait(self.browser, 10)
        self.todos_os_produtos = []

        print(f"Acessando a URL base: {self.base_url}\n")
        self.browser.get(self.base_url)
        self.aceitar_cookies()

    def aceitar_cookies(self):
        try:
            cookie_wait = WebDriverWait(self.browser, 4)
            cookie_wait.until(EC.visibility_of_element_located((By.CLASS_NAME, classe_caixa_aceitar)))
            cookie_wait.until(EC.element_to_be_clickable((By.ID, id_aceitar))).click()
            print('Cookies aceitos com sucesso!\n')
        except TimeoutException:
            print("Banner de cookies não encontrado ou já foi aceito.\n")

    def scroll_e_clica_js(self, xpath_alvo):
        try:
            elemento = self.wait.until(EC.presence_of_element_located((By.XPATH, xpath_alvo)))
            self.browser.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", elemento)
            time.sleep(1)
            self.browser.execute_script("arguments[0].click();", elemento)
            return True
        except TimeoutException:
            print(f"ERRO: Elemento não encontrado com o XPath: {xpath_alvo}")
            return False

    
    def raspagem_da_pagina(self, filtro_aplicado):
        print(f"   Iniciando a raspagem para o filtro '{filtro_aplicado}'...")
        try:
            produtos = self.wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, CLASSE_PRODUTO_ITEM)))
            print(f"   Encontrados {len(produtos)} produtos.")
            if not produtos: return

            for produto in produtos:
                try:
                    nome = produto.find_element(By.CLASS_NAME, CLASSE_PRODUTO_NOME).text
                    link = produto.find_element(By.CLASS_NAME, CLASSE_PRODUTO_NOME).get_attribute('href')
                    preco = produto.find_element(By.CLASS_NAME, CLASSE_PRODUTO_PRECO).text
                    descricao = produto.find_element(By.CLASS_NAME, CLASSE_PRODUTO_DESCRICAO).text
                    
                    self.todos_os_produtos.append({
                        'Filtro Aplicado': f'Painel {filtro_aplicado}',
                        'Nome do Produto': nome,
                        'Descrição': descricao.replace("\n", " "),
                        'Preço': preco.replace("\n", " "),
                        'Link': link
                    })
                except (StaleElementReferenceException, NoSuchElementException):
                    print("   - Aviso: Um produto foi ignorado (dados incompletos ou referência obsoleta). Continuando...")
                    continue
        except TimeoutException:
            print("Nenhum produto encontrado para raspar.")


    def salvar_dados(self, nome_arquivo="extracao_neosolar_completa.xlsx"):
        if not self.todos_os_produtos:
            print("\nNenhum dado foi coletado para salvar.")
            return
        print(f"\nSalvando {len(self.todos_os_produtos)} produtos em '{nome_arquivo}'...")
        df = pd.DataFrame(self.todos_os_produtos)
        df.to_excel(nome_arquivo, index=False, engine='openpyxl')
        print(f"Arquivo '{nome_arquivo}' salvo com sucesso!")

    def fechar(self):
        if self.browser:
            print("Fechando o navegador.")
            self.browser.quit()

    def extracao_offgrid(self):
        self.link_offgrid = 'https://www.neosolar.com.br/loja/kit-energia-solar-off-grid.html'
        self.browser.get(self.link_offgrid)
        print(f"Acessando a URL para descobrir filtros: {self.link_offgrid}\n")

        XPATH_TITULO_POTENCIA_PAINEL = '//div[@class="filter-options-title" and contains(text(), "Potência do Painel")]'
        XPATH_OPCOES_POTENCIA_PAINEL = '//div[contains(text(), "Potência do Painel")]/following-sibling::div//a'

        print("1. Descobrindo os links diretos para cada filtro...")
        if not self.scroll_e_clica_js(XPATH_TITULO_POTENCIA_PAINEL):
            print("Falha ao expandir o filtro 'Potência do Painel'. Abortando.")
            return
        
        links_de_filtro = []
        try:
            opcoes_elementos = self.wait.until(EC.presence_of_all_elements_located((By.XPATH, XPATH_OPCOES_POTENCIA_PAINEL)))
            for elem in opcoes_elementos:
                label_limpo = elem.text.split('\n')[0].strip()
                link = elem.get_attribute('href')
                if label_limpo and link:
                    links_de_filtro.append({'label': label_limpo, 'url': link})
            
            if not links_de_filtro:
                print("Nenhuma opção de filtro encontrada. Abortando.")
                return
            print(f"Opções e links coletados: {links_de_filtro}\n")
        except TimeoutException:
            print("Não foi possível encontrar a lista de opções de filtro. Abortando.")
            return

        for filtro in links_de_filtro:
            label = filtro['label']
            url = filtro['url']
            
            print(f"--- Processando filtro para: Painel {label} ---")
            print(f"   Navegando diretamente para: {url}")
            self.browser.get(url)
            
            self.raspagem_da_pagina(label)
            print("-" * 50)

if __name__ == "__main__":
    automacao = None
    try:
        automacao = NEO()
        automacao.extracao_offgrid()
        automacao.salvar_dados()
        print("\nAutomação concluída com sucesso.")
    except Exception as e:
        print(f"Ocorreu um erro fatal na execução do robô: {e}")
    finally:
        if automacao:
            automacao.fechar()
        print("\nScript finalizado.")

Acessando a URL base: https://www.neosolar.com.br/loja

Cookies aceitos com sucesso!

Acessando a URL para descobrir filtros: https://www.neosolar.com.br/loja/kit-energia-solar-off-grid.html

1. Descobrindo os links diretos para cada filtro...
Opções e links coletados: [{'label': '30W', 'url': 'https://www.neosolar.com.br/loja/kit-energia-solar-off-grid.html?b1_potencia_do_painel=4912'}, {'label': '60W', 'url': 'https://www.neosolar.com.br/loja/kit-energia-solar-off-grid.html?b1_potencia_do_painel=4913'}, {'label': '100W', 'url': 'https://www.neosolar.com.br/loja/kit-energia-solar-off-grid.html?b1_potencia_do_painel=4914'}, {'label': '160W', 'url': 'https://www.neosolar.com.br/loja/kit-energia-solar-off-grid.html?b1_potencia_do_painel=4936'}, {'label': '200W', 'url': 'https://www.neosolar.com.br/loja/kit-energia-solar-off-grid.html?b1_potencia_do_painel=4937'}, {'label': '620W', 'url': 'https://www.neosolar.com.br/loja/kit-energia-solar-off-grid.html?b1_potencia_do_painel=4994'}]

--- 